In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 32.2 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 346 kB 57.2 MB/s 
     |████████████████████████████████| 1.2 MB 58.0 MB/s 
     |████████████████████████████████| 6.6 MB 56.9 MB/s 
     |████████████████████████████████| 1.8 MB 62.1 MB/s 
     |████████████████████████████████| 10.1 MB 55.7 MB/s 
     |████████████████████████████████| 4.2 MB 55.6 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 145 kB 67.9 MB/s 
     |████████████████████████████████| 181 kB 27.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 86 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 30.0 MB/s 
     |████████████████████████████████

In [ ]:
import pandas as pd 

df = pd.read_csv("/content/drive/MyDrive/Major Project/Bert/DataSet.csv", header=0, names=['review', 'sentiment'])
print(df.head())

                                              review sentiment
0  गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...  negative
1  ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है  negative
2  यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...  negative
3  अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...  negative
4                    कश्मीर में हो रहा है जल जिहाद ।  negative


In [ ]:
print(set(df['sentiment'].values))

{'positive', 'neutral', 'negative'}


In [ ]:
df.sentiment = pd.Categorical(df.sentiment)
df['sentiment'] = df['sentiment'].cat.codes
df = df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=91)

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:

bert = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 10,
    'silent': True
})
bert.train_model(train)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


(8510, 0.35204863191124536)

In [ ]:
result, model_outputs, wrong_predictions = bert.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


wrong predictions
442 wrong out of 2269


{0: 193, 1: 51, 2: 198}

In [ ]:
(1 - (442 / 2269)) * 100

80.52005288673425

80.5% @ 10 epochs
